In [110]:
import os
import random
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.svm import SVR, SVC

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import lightgbm as lgb
from random import randint

In [13]:
tolerate = {"AGC":{"AGC_A", "AGC_B"}, 
            "LSIL":{"ASCUS", "LSIL_E", "LSIL_F"}, 
            "ASCUS":{"ASCUS", "LSIL_E", "LSIL_F"}, 
            "HSIL-SCC_G":{"HSIL_B", "HSIL_M", "HSIL_S", "SCC_G"}, 
            "SCC_R":{"SCC_R"}, 
            "EC":{"EC"}, 
            "CC":{"CC"}, 
            "VIRUS":{"VIRUS"}, 
            "FUNGI":{"FUNGI"}, 
            "ACTINO":{"ACTINO"}, 
            "TRI":{"TRI"}, 
            "PH":{"PH"}, 
            "SC":{"SC", "RC", "MC", "GEC"}}

bin_imap = {0:'NILM', 1:'ABN'}
all_imap = {0: 'ACTINO', 1: 'AGC', 2: 'ASCH', 3: 'ASCUS', 4: 'CC', 5: 'EC', 6: 'CANDIDA', 
            7: 'HSIL', 8: 'HSV', 9: 'LSIL', 10: 'NILM', 11: 'SCC', 12: 'TRI'}



In [173]:
# detect:0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9; 
# classify: 0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99,0.999

# with open('train15test1500.pkl', 'wb') as f:
#     pickle.dump(X, f)
#     pickle.dump(ya, f)
#     pickle.dump(yb, f)
#     pickle.dump(names, f)


## 1. get data
with open('gnet2data2800.pkl', 'rb') as f:
    X = pickle.load(f)
    ya = pickle.load(f)
    yb = pickle.load(f)
    names = pickle.load(f)
    
with open('header_imap.pkl', 'rb') as f:
    idx = pickle.load(f)

## 2. Do data augmentation

# 2.1 environment setup

label_cell = {'ACTINO':'ACTINO','AGC':'AGC','ASCUS':'ASCUS','CC':'CC','EC':'EC',
              'CANDIDA':'FUNGI','HSIL':'HSIL-SCC_G','ASCH':'HSIL-SCC_G','LSIL':'LSIL',
              'PH':'PH','NILM':'SC','SCC':'SCC_R','TRI':'TRI','HSV':'VIRUS'}

label = ['ACTINO','AGC', 'ASCH', 'ASCUS','CC', 'EC', 'CANDIDA',
            'HSIL', 'HSV','LSIL', 'NILM', 'SCC', 'TRI']


det_score = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
cls_score = [0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99,0.999]
len_det_score = len(det_score)
len_cls_score = len(cls_score) 
# _, cnts = np.unique(ya,return_counts=True)
# print(cnts)
# print(idx[(label[11], det_score[5], cls_score[8])])

# 2.2 construct feature dictionary.
feature_dict = {}

for i in range(len(X)):
    key = label[ya[i]]
    if key in feature_dict:
        feature_dict[key].append(X[i])
    else:
        feature_dict[key] = [X[i]]

# 2.3 augmentation for one feature.
def feature_aug(feature, key):
    inc_num = randint(-2,8)
    dec_num = randint(-8,2)
    det_pos = randint(int(len(det_score)/2), len(det_score)-1)
    cls_pos = randint(int(len(det_score)/2), len(det_score)-1)
    if key == 'ASCH':
#         inc_num  //= 5
        pass
    for i in range(det_pos)[::-1]:
        for j in range(cls_pos)[::-1]:
            cur_pos = idx[(key, det_score[i],cls_score[j])]
            feature[cur_pos] += inc_num
            feature[cur_pos+13*90] += inc_num
            inc_num += randint(0,3)
            for k in label:
                if k == key:
                    pass
                cur_pos = idx[(key, det_score[i], cls_score[j])]
                feature[cur_pos] = max(0, feature[cur_pos]+dec_num)
                feature[cur_pos+13*90] = max(0, feature[cur_pos+13*90]+dec_num)
    return feature
            
# 2.4 augmentation for all features.    
aug_num_per_cls = 1500
for key in feature_dict:
    features = np.array(feature_dict[key])
    feat_rows, feat_cols = features.shape
    print(feat_rows, feat_cols)
    for aug_num in range(aug_num_per_cls-feat_rows):
        feature = features[randint(0,1000)%feat_rows]
        feat_aug = feature_aug(feature, label_cell[key])
        feature_dict[key].append(feat_aug)

# for check 
for key in feature_dict:
    features = np.array(feature_dict[key])
    print(features.shape)


349 2340
34 2340
37 2340
11 2340
300 2340
69 2340
80 2340
42 2340
118 2340
921 2340
236 2340
175 2340
413 2340
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)
(1500, 2340)


In [174]:
with open("feature_dict.pkl", 'wb') as f:
    pickle.dump(feature_dict, f)

In [171]:
for i in range(10)[::-1]:
    print(i)

9
8
7
6
5
4
3
2
1
0
